In [1]:
import pandas as pd
import numpy as np
import ast
import seaborn
from collections import Counter
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer("english")
from bs4 import BeautifulSoup
import logging, gensim, bz2
import os
import re
from operator import itemgetter

/Users/YJChang/anaconda/envs/python2/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
def load(filename):
    file = open(filename,'rb')
    obj = pickle.load(file)
    file.close()
    return obj

def dump(obj,filename):
    filehandler = open(filename,"wb")
    pickle.dump(obj,filehandler)
    filehandler.close()

In [6]:
business_data = pd.read_csv("../data/yelp_academic_dataset_business.csv")

In [7]:
cities = set(business_data["city"])
cities

{nan,
 'AGINCOURT',
 'Aberdour',
 'Aberlady',
 'Ahwahtukee',
 'Ahwatukee',
 'Ahwatukee Foothills Village',
 'Aichwald',
 'Ajax',
 'Alburg',
 'Allegheny',
 'Allison Park',
 'Ambridge',
 'Amherst',
 'Anjou',
 'Ansnorveldt',
 'Anthem',
 'Arlington',
 'Arnold',
 'Aspinwall',
 'Auburn',
 'Auburn Township',
 'Aurora',
 'Avalon',
 'Avon',
 'Avon Lake',
 'Avondale',
 "Baie-D'urfe",
 "Baie-d'Urf\xc3\xa9",
 'Bainbridge',
 'Bainbridge Township',
 'Baldwin',
 'Balerno',
 'Ballantyne',
 'Banksville',
 'Bath',
 'Bathurst Quay',
 'Bay Village',
 'Beachwood',
 'Beaconsfield',
 'Bedford',
 'Bedford HTS',
 'Bedford Heights',
 'Bedford Hts.',
 'Beeton',
 'Belleville',
 'Bellevue',
 'Bellvue',
 'Belmont',
 'Beloeil',
 'Ben Avon',
 'Bentleyville',
 'Berea',
 'Berg',
 'Bernhausen',
 'Berry',
 'Bethel Park',
 'Black Earth',
 'Blackness',
 'Blainville',
 'Blawnox',
 'Bloomfield',
 'Blue Diamond',
 'Bois-Des-Filion',
 'Boisbriand',
 'Bolton',
 'Bonnyrigg',
 'Bonnyrigg and Lasswade',
 'Boston Heights',
 'Bouche

In [13]:
business_data.head(n=8)

,neighborhood,business_id,hours,is_open,address,attributes,categories,city,review_count,name,longitude,state,stars,latitude,postal_code,type
0,NaN,0DI8Dt2PJp07XkVvIElIcQ,"['Monday 11:0-21:0', 'Tuesday 11:0-21:0', 'Wed...",0,"227 E Baseline Rd, Ste J2","['BikeParking: True', 'BusinessAcceptsBitcoin:...","['Tobacco Shops', 'Nightlife', 'Vape Shops', '...",Tempe,17,Innovative Vapors,-111.936102,AZ,4.5,33.378214,85283,business
1,NaN,LTlCaCGZE14GuaUXUGbamg,"['Monday 0:0-0:0', 'Tuesday 0:0-0:0', 'Wednesd...",1,495 S Grand Central Pkwy,"['BusinessAcceptsBitcoin: False', 'BusinessAcc...","['Caterers', 'Grocery', 'Food', 'Event Plannin...",Las Vegas,9,Cut and Taste,-115.159272,NV,5.0,36.192284,89106,business
2,Dufferin Grove,EDqCEAGXVGCH4FJXgqtjqg,"['Monday 11:0-2:0', 'Tuesday 11:0-2:0', 'Wedne...",1,979 Bloor Street W,"['Alcohol: none', ""Ambience: {'romantic': Fals...","['Restaurants', 'Pizza', 'Chicken Wings', 'Ita...",Toronto,7,Pizza Pizza,-79.429089,ON,2.5,43.661054,M6H 1L5,business
3,NaN,cnGIivYRLxpF7tBVR_JwWA,"['Tuesday 10:0-21:0', 'Wednesday 10:0-21:0', '...",1,7014 Steubenville Pike,"['AcceptsInsurance: False', 'BusinessAcceptsCr...","['Hair Removal', 'Beauty & Spas', 'Blow Dry/Ou...",Oakdale,4,Plush Salon and Spa,-80.174540,PA,4.0,40.444544,15071,business
4,Downtown Core,cdk-qqJ71q6P7TJTww_DSA,NaN,1,321 Jarvis Street,"['BusinessAcceptsCreditCards: True', 'Restaura...","['Hotels & Travel', 'Event Planning & Services...",Toronto,8,Comfort Inn,-79.375401,ON,3.0,43.659829,M5B 2C2,business
5,NaN,Q9rsaUiQ-A3NdEAloy0aJA,"['Monday 10:30-20:0', 'Tuesday 10:30-20:0', 'T...",1,"30 Gibson Drive, Suite122","['BusinessAcceptsCreditCards: True', 'ByAppoin...","['Nail Salons', 'Beauty & Spas']",Markham,3,A Plus Nail,-79.342722,ON,2.5,43.827640,L3R 2S3,business
6,NaN,Cu4_Fheh7IrzGiK-Pc79ig,"['Monday 10:0-18:0', 'Tuesday 10:0-18:0', 'Wed...",1,10875 N Frankloyd Wright Blvd,"['BikeParking: True', 'BusinessAcceptsCreditCa...","['Baby Gear & Furniture', 'Shopping']",Scottsdale,8,Boomerang Baby,-111.834954,AZ,3.5,33.585271,85259,business
7,NaN,GDnbt3isfhd57T1QqU6flg,"['Monday 10:0-22:0', 'Tuesday 10:0-22:0', 'Wed...",1,11072 No Frank Lloyd Wright,"['Alcohol: none', ""Ambience: {'romantic': Fals...","['Tex-Mex', 'Mexican', 'Fast Food', 'Restauran...",Scottsdale,9,Taco Bell,-111.835410,AZ,2.5,33.586710,85259,business


In [20]:
IsDuplicated = business_data.duplicated(['business_id'])  
IsDuplicated.value_counts()

False    144072
dtype: int64

In [22]:
#pitt_business_data = business_data[business_data["city"]=="Pittsburgh"]
montreal_business_data_1 = business_data[business_data["city"]=="Montreal"]
montreal_business_data_2 = business_data[business_data["city"]=="Montreal-Nord"]
montreal_business_data_3 = business_data[business_data["city"]=="Montreal-Ouest"]
montreal_business_data_4 = business_data[business_data["city"]=="Montreal-West"]
montreal_business_data_5 = business_data[business_data["city"]=="Montr\xc3\xa9al"]
montreal_business_data_6 = business_data[business_data["city"]=="Montr\xc3\xa9al-Nord"]
montreal_business_data_7 = business_data[business_data["city"]=="Montr\xc3\xa9al-Ouest"]
montreal_business_data_8 = business_data[business_data["city"]=="Mont\xc3\xa9al"]


In [23]:
montreal_business_data = pd.concat([montreal_business_data_1, montreal_business_data_2, montreal_business_data_3, montreal_business_data_4, montreal_business_data_5, montreal_business_data_6, montreal_business_data_7, montreal_business_data_8])

In [72]:
#pitt_business_data.head(n=5)

,neighborhood,business_id,hours,is_open,address,attributes,categories,city,review_count,name,longitude,state,stars,latitude,postal_code,type
38,West View,P3LisOj7DktgGa7C5FYpnA,"['Monday 9:30-9:0', 'Tuesday 9:30-9:0', 'Wedne...",1,451 Rochester Rd,['BusinessAcceptsCreditCards: True'],"['Plumbing', 'Home Services']",Pittsburgh,8,Benjamin Franklin Plumbing,-80.049887,PA,4.0,40.534163,15237,business
44,North Side,93otbGHE0s0m-lU1osvg9w,NaN,1,337 N Shore Dr,"['Alcohol: full_bar', ""Ambience: {'romantic': ...","['American (New)', 'Pubs', 'Nightlife', 'Bars'...",Pittsburgh,102,Rivertowne,-80.010880,PA,3.0,40.445986,15212,business
56,Greentree,csdZolWIWvkIHIqsCmV6sg,"['Monday 10:0-18:0', 'Tuesday 10:0-18:0', 'Wed...",1,325 Elizabeth Dr,"['BusinessAcceptsCreditCards: True', 'ByAppoin...","['Hotels & Travel', 'Home Services', 'Hotels',...",Pittsburgh,5,City Vista,-80.037872,PA,2.0,40.426386,15220,business
81,Bloomfield,h7ZW6c1Vi9V6_-gd2Ch_4g,"['Monday 10:0-21:0', 'Tuesday 10:0-21:0', 'Wed...",1,4815 Centre Ave,"['BusinessAcceptsCreditCards: True', ""Business...","['Shopping', 'Mobile Phones']",Pittsburgh,5,Verizon Wireless Premium Retailer,-79.946995,PA,2.0,40.453725,15213,business
107,Strip District,HWrbZS1mxVRj2Y2VwMmDMg,NaN,0,2627 Penn Ave,"['BikeParking: True', 'BusinessAcceptsCreditCa...","['Food', 'Bakeries']",Pittsburgh,3,Dulcinea Bakeshop,-79.977728,PA,4.5,40.455351,15222,business


In [67]:
#pitt_review.head(n=5)

,funny,user_id,review_id,text,business_id,stars_x,date,useful,type_x,cool,...,review_count,name,longitude,state,stars_y,latitude,postal_code,type_y,Restaurant,clean_text
1032,1,-UPUYET3Pwm99zq-uHl7gg,OPBHCs-lcEVWbEZY7rKNUg,I would give this a 3 1/2. Typical bar food s...,0Wy4gw8krao9nGq-sHWFSQ,3,2013-12-01,1,review,1,...,41,Brewstone,-79.809634,PA,2.5,40.429825,15235,business,1,"[would, give, typic, bar, food, select, food, ..."
1033,0,WLrFY6_Z32lb_ifcIvRvkw,SjgayD75MU7yTWEs_cGbwg,It was hit or miss for us. \n\nWe got burgers ...,0Wy4gw8krao9nGq-sHWFSQ,3,2013-05-06,0,review,0,...,41,Brewstone,-79.809634,PA,2.5,40.429825,15235,business,1,"[hit, miss, u, got, burger, first, bread, look..."
1034,0,pOQduF1Wf8fQeXlWEmNH2w,0cBOrzOUqdm-eZHQJDbKAA,My girlfriend and I showed up at 7:30 on a Fri...,0Wy4gw8krao9nGq-sHWFSQ,1,2013-06-04,0,review,0,...,41,Brewstone,-79.809634,PA,2.5,40.429825,15235,business,1,"[girlfriend, show, 30, friday, night, complet,..."
1035,0,UyhtAbEnjTtp-v0CQneh6Q,WY3Bl7Huzv6Bzg-gMmnyMg,Absolutely terrible! Just got up and left...se...,0Wy4gw8krao9nGq-sHWFSQ,1,2014-09-08,0,review,0,...,41,Brewstone,-79.809634,PA,2.5,40.429825,15235,business,1,"[absolut, terribl, got, left, seat, 15, minut,..."
1036,0,3QHmbVzpWPM2PKm4kAI01Q,vPcoXzJbA9U77RYkPwxXbw,"I had heard some bad reviews about Brewstone, ...",0Wy4gw8krao9nGq-sHWFSQ,4,2012-08-12,2,review,0,...,41,Brewstone,-79.809634,PA,2.5,40.429825,15235,business,1,"[heard, bad, review, brewston, especi, servic,..."


In [3]:
review_with_topics = load("montreal_reviews_with_topics.pkl")

In [4]:
review_with_topics.head(n=10)

,index,user_id,review_id,text,name,attributes,categories,stars_x,stars_y,Top_ID,Top_name,Top_level2,Score_top,All_ID,All_name,All_level2,All_score,Top_Topic_Words
0,3337,ffN1CWNbE9yJhRBobTGzXg,0RMyKgb9wmhvJeRiXkQ0Rw,My go to spot in Chinatown after Van Roy's had...,New Dynasty,"['Alcohol: beer_and_wine', ""Ambience: {'romant...","[Chinese, Restaurants]",5,4.0,22,food_quality,perfect_food,0.389905,"[22, 6, 39, 12, 38]","[food_quality, service, cusine, service, Needs...","[perfect_food, time, thai, pay, ]","[(22, 0.389904908016), (6, 0.349042460405), (3...","[montreal, best, place, one, time, ever, visit..."
1,3338,WRUJyTucViNteOoS8lvcvQ,lsdB5G8UxczlGRy16VfAJg,"Un restaurant où l'on mange vraiment chinois, ...",New Dynasty,"['Alcohol: beer_and_wine', ""Ambience: {'romant...","[Chinese, Restaurants]",5,4.0,17,NeedsLabelling,,0.568653,"[17, 8, 43, 6]","[NeedsLabelling, NeedsLabelling, NeedsLabellin...","[, , , time]","[(17, 0.568652610241), (8, 0.306799682846), (4...","[le, et, de, est, un, dã, pour, la, trã, bon, ..."
2,3339,JLdxmsngE7cjrwNT7zqgyQ,zOeQKVeJXjcii1uQibnpHA,I wanted to check this out since a fellow frie...,New Dynasty,"['Alcohol: beer_and_wine', ""Ambience: {'romant...","[Chinese, Restaurants]",3,4.0,35,food_quality,food,0.355489,"[35, 23, 18, 25, 32, 40, 48, 38, 47, 0, 19, 13]","[food_quality, service, NeedsLabelling, cusine...","[food, waiting_time_seat, , chicken, seat, wai...","[(35, 0.35548900761), (23, 0.176585947482), (1...","[food, like, good, place, tast, would, realli,..."
3,3340,D54H8qi-1g21imV39w4Gzg,tIwgCfqDv8WhVmakwQ8OmQ,The place is run down but the food is deliciou...,New Dynasty,"['Alcohol: beer_and_wine', ""Ambience: {'romant...","[Chinese, Restaurants]",5,4.0,22,food_quality,perfect_food,0.910909,[22],[food_quality],[perfect_food],"[(22, 0.910909090906)]","[montreal, best, place, one, time, ever, visit..."
4,3341,LuYIMPQr0IeLVj2AQ36dzw,P89kJDM8dv9j0-4CrOZ_-w,This place has some of the best Chinese food i...,New Dynasty,"['Alcohol: beer_and_wine', ""Ambience: {'romant...","[Chinese, Restaurants]",4,4.0,33,cusine,salad,0.218984,"[33, 44, 22, 38, 35, 6, 5, 24, 13]","[cusine, food_quality, food_quality, NeedsLabe...","[salad, food_good, perfect_food, , food, time,...","[(33, 0.21898363156), (44, 0.212912645788), (2...","[salad, sauc, fresh, perfect, bread, tomato, s..."
5,3342,wlLjTKFAkwYNuGd8b7Rwbw,xjOb9HSIuZCPwjZn1RSxJQ,The New Dynasty is one street west of St-Laure...,New Dynasty,"['Alcohol: beer_and_wine', ""Ambience: {'romant...","[Chinese, Restaurants]",3,4.0,35,food_quality,food,0.214664,"[35, 48, 25, 38, 40, 29, 36, 22, 13, 31, 37, 9...","[food_quality, NeedsLabelling, cusine, NeedsLa...","[food, , chicken, , wait, , place, perfect_foo...","[(35, 0.214663785843), (48, 0.152156934548), (...","[food, like, good, place, tast, would, realli,..."
6,3343,orBnbkHheyQf2hWKzH8FyA,vkAmtASHo_ap5a9YOnoOow,I haven't been to china so I can't say for sur...,New Dynasty,"['Alcohol: beer_and_wine', ""Ambience: {'romant...","[Chinese, Restaurants]",4,4.0,18,NeedsLabelling,,0.352558,"[18, 25, 38, 35, 34, 40]","[NeedsLabelling, cusine, NeedsLabelling, food_...","[, chicken, , food, price, wait]","[(18, 0.352557769386), (25, 0.181527872922), (...","[great, food, place, servic, good, friend, del..."
7,3344,WnDuJbLhJkLo1I_--63vFw,6-oSvb0sZN7R2yh_UkP6NA,Note: go there late at night if you can. The d...,New Dynasty,"['Alcohol: beer_and_wine', ""Ambience: {'romant...","[Chinese, Restaurants]",4,4.0,18,NeedsLabelling,,0.183032,"[18, 25, 22, 33, 44, 6, 35, 42, 5, 38, 30, 1, ...","[NeedsLabelling, cusine, food_quality, cusine,...","[, chicken, perfect_food, salad, food_good, ti...","[(18, 0.183031574492), (25, 0.140606752476), (...","[great, food, place, servic, good, friend, del..."
8,3345,ReDypdRmVA3wc-QG8HBKkQ,uTj4eOFvswqpufKQy1z8eQ,I'm writing this review in light of the YouTub...,New Dynasty,"['Alcohol: beer_and_wine', ""Ambience: {'romant...","[Chinese, Restaurants]",4,4.0,22,food_quality,perfect_food,0.420276,"[22, 

In [24]:
simp_rate_top = review_with_topics.groupby(["name","Top_name"])["stars_x"].mean()
simp_rate_top = simp_rate_top.unstack()
simp_rate_top = pd.merge(simp_rate_top, montreal_business_data[["name","stars"]],left_index=True,right_on="name")

In [25]:
cols = simp_rate_top.columns.tolist()
cols = cols[-2:] + cols[:-2]
simp_rate_top = simp_rate_top[cols]

In [26]:
simp_rate_top.to_csv("../Pickled_files/montreal_simp_rate_top.csv")
dump(simp_rate_top,"../Pickled_files/montreal_simp_rate_top.pkl")

## Trying different method

In [27]:
list_dict = map(lambda x: Counter(x),review_with_topics["All_name"])

In [28]:
topic_cat = pd.DataFrame(list_dict)

In [29]:
simp_rate_all = pd.concat([review_with_topics[["review_id","name","stars_x"]],topic_cat],axis=1)
simp_rate_all = pd.melt(simp_rate_all,id_vars=["review_id","name","stars_x"],value_vars=topic_cat.columns.tolist())
simp_rate_all = simp_rate_all[simp_rate_all["value"].notnull()]
simp_rate_all = simp_rate_all.groupby(["name","variable"])["stars_x"].mean()
simp_rate_all= simp_rate_all.unstack()
simp_rate_all = pd.merge(simp_rate_all,montreal_business_data[["name","stars"]],left_index=True,right_on="name")

In [151]:
cols = simp_rate_all.columns.tolist()
cols = cols[-2:] + cols[:-2]
simp_rate_all = simp_rate_all[cols]

In [30]:
simp_rate_all.to_csv("../Pickled_files/montreal_simp_rate_all.csv")
dump(simp_rate_all,"../Pickled_files/montreal_simp_rate_all.pkl")